<a href="https://colab.research.google.com/github/Priyansh-Kedia/AmazonChallenge/blob/master/AmazonChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# Add all imports here

from nltk.corpus import stopwords
import nltk
import string
import re

In [14]:
# Download dataset

!wget https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip


--2021-07-30 05:44:01--  https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.125.22
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.125.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1061576029 (1012M) [binary/octet-stream]
Saving to: ‘dataset52a7b21.zip.3’

dataset52a7b21.zip.   0%[                    ]  49.59K   144KB/s               ^C
/bin/bash: -c: line 0: syntax error near unexpected token `'stopwords''
/bin/bash: -c: line 0: `nltk.download('stopwords')'


In [ ]:
# Unzip the dataset

!unzip dataset52a7b21.zip

Archive:  dataset52a7b21.zip
   creating: dataset/
  inflating: dataset/train.csv       
  inflating: dataset/sample_submission.csv  
  inflating: dataset/test.csv        
  inflating: dataset/.~lock.train.csv#  


In [19]:
# Download nltk stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Import the dataset in pandas

import pandas as pd
import csv
df = pd.read_csv("dataset/train.csv", low_memory=True, escapechar="\\", quoting=csv.QUOTE_NONE)

In [ ]:
# Add the column variables here

TITLE = "TITLE"
PRODUCT_ID = "PRODUCT_ID"
DESCRIPTION = "DESCRIPTION"
BULLET_POINTS = "BULLET_POINTS"
BRAND = "BRAND"
BROWSE_NODE_ID = "BROWSE_NODE_ID"

In [ ]:
# Load the doc
def load_doc(df):
  titles = df[TITLE].tolist()
  descriptions = df[DESCRIPTION].tolist()
  bullets = df[BULLET_POINTS].tolist()
  

load_doc(df)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [34]:
# To remove html tags

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [33]:
def clean_doc(doc):
  doc = cleanhtml(doc)
  tokens = doc.split() # Add delimiter if needed

  punctuations = str.maketrans('','', string.punctuation)

  tokens = [w.translate(punctuations) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  stop_words = set(stopwords.words("english"))
  tokens = [w for w in tokens if not w in stop_words]

  tokens = [word for word in tokens if len(word) > 1]

  return tokens


In [35]:
# This is just created to test the above functions

tokens = clean_doc('testing this for the use case <p>this should also be tested</p>')
print(tokens)

['testing', 'use', 'case', 'also', 'tested']
